In [24]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html

# Import the API key.
from config import g_key

In [25]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Birao,10.2849,22.7882,71.67,90,20,3.47,CF,2022-10-06 00:00:43
1,1,Rikitea,-23.1203,-134.9692,68.97,66,12,21.54,PF,2022-10-06 00:00:43
2,2,Hamilton,39.1834,-84.5333,66.04,56,41,2.66,US,2022-10-05 23:58:50
3,3,Vaini,-21.2000,-175.2000,76.12,85,100,10.02,TO,2022-10-06 00:00:44
4,4,Busselton,-33.6500,115.3333,56.01,57,14,12.50,AU,2022-10-06 00:00:44


In [26]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [27]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [28]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [29]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig
embed_minimal_html('heatmap_temp.html', views=[fig])

In [30]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig
embed_minimal_html('heatmap_humidity.html', views=[fig])

In [31]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig
embed_minimal_html('heatmap_humidity_2.html', views=[fig])

In [34]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [35]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Vaini,-21.2000,-175.2000,76.12,85,100,10.02,TO,2022-10-06 00:00:44
7,7,Georgetown,5.4112,100.3354,77.90,91,20,1.14,MY,2022-10-06 00:00:27
25,25,Santa Isabel Do Rio Negro,-0.4139,-65.0192,79.66,73,46,3.65,BR,2022-10-06 00:00:51
28,28,Sinnamary,5.3833,-52.9500,79.81,84,3,11.10,GF,2022-10-05 23:59:12
34,34,Bathsheba,13.2167,-59.5167,81.21,83,75,14.97,BB,2022-10-05 23:59:07
39,39,Hilo,19.7297,-155.0900,82.27,79,100,8.05,US,2022-10-05 23:57:34
48,48,Gizo,-8.1030,156.8419,84.27,66,77,7.56,SB,2022-10-05 23:54:01
49,49,Waingapu,-9.6567,120.2641,82.90,72,98,9.73,ID,2022-10-06 00:01:57
54,54,Atuona,-9.8000,-139.0333,77.61,69,4,17.29,PF,2022-10-06 00:01:58
57,57,Acapulco,16.8634,-99.8901,81.55,81,43,3.47,MX,2022-10-06 00:00:51


In [36]:
preferred_cities_df.count()

City_ID       13
City          13
Lat           13
Lng           13
Max Temp      13
Humidity      13
Cloudiness    13
Wind Speed    13
Country       13
Date          13
dtype: int64

In [37]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Vaini,TO,76.12,-21.2000,-175.2000,
7,Georgetown,MY,77.90,5.4112,100.3354,
25,Santa Isabel Do Rio Negro,BR,79.66,-0.4139,-65.0192,
28,Sinnamary,GF,79.81,5.3833,-52.9500,
34,Bathsheba,BB,81.21,13.2167,-59.5167,
39,Hilo,US,82.27,19.7297,-155.0900,
48,Gizo,SB,84.27,-8.1030,156.8419,
49,Waingapu,ID,82.90,-9.6567,120.2641,
54,Atuona,PF,77.61,-9.8000,-139.0333,
57,Acapulco,MX,81.55,16.8634,-99.8901,


In [38]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [39]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [40]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig
embed_minimal_html('heatmap_vacation_spots.html', views=[fig])

In [41]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
embed_minimal_html('heatmap_temperature_spots_2.html', views=[fig])

In [42]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [43]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
embed_minimal_html('heatmap_final.html', views=[fig])